In [68]:
Hateful = {}

do
    local constructor
    constructor = function(T, file)
        local cfg = {__type = T, file = file}
        cfg.write = function(self)
            local f = io.open(self.file)
            f:write(Hateful.dump(obj))
            f:close()
            return self
        end
        cfg.read = function(self)
            require(file:gsub(".lua", ""))
        end
        return cfg
    end
    Hateful.Config = setmetatable({
            __type = "Type",
            name = "Hateful.Config",
            abstract = false,
        }, {
            __call = constructor
        })
end

Hateful.fromfile = function()
    return {
        __type = "Hateful.Config",
        file = file
    }
end

Hateful.dump = function(self, table, level, shown)
    shown = shown or {}
    level = level or 1
    if level == 1 then
        shown[self] = true
    end
    -- doesn't handle cyclic reference
    if level > 100 then
        return "Overflow"
    end
    local ret = ""
    local keypart, valuepart
    local auxstart, auxend = "", ""
    local indent = "\t"
    for k, v in pairs(table) do
        if type(k) == "number" or type(k) == "boolean" then
            keypart = string.format("[%s]", k)
        elseif type(k) == "string" then
            keypart = tostring(k)
        else
            error("Key as other data type is not supported")
        end
        if type(v) == "string" then
            valuepart = string.format([["%s"]], v)
        elseif type(v) == "table" then
            print(v, shown[v])
            if shown[v] then
                valuepart = tostring(v)
            else
                shown[v] = true
                valuepart = self:dump(v, level + 1, shown)
            end
        else
            valuepart = v
        end
        ret = ret .. "\n" ..
            string.format("%s%s = %s,", indent:rep(level), keypart, valuepart)
    end
    return string.format("{%s\n%s}", ret, indent:rep(level - 1))
end

In [69]:
t = {
    rice = "Openbox-based",
    scheme = "Fleon",
    params = {
        terminal = "urxvt",
        browser = "firefox",
        ntags = 5
    },
    123,
    [true] = 1
}
t.self = t

In [60]:
cfg = Hateful.Config()
print(cfg.__type.name)

Hateful.Config


In [73]:
print(dump_raw(t))

table: 0x558fc079a8e0
  1 : 123 (number)
  params : table: 0x558fc079a920
    terminal : urxvt (string)
    ntags : 5 (number)
    browser : firefox (string)
  rice : Openbox-based (string)
  scheme : Fleon (string)
  self : table: 0x558fc079a8e0
    1 : 123 (number)
    params : table: 0x558fc079a920
      terminal : urxvt (string)
      ntags : 5 (number)
      browser : firefox (string)
    rice : Openbox-based (string)
    scheme : Fleon (string)
    self : table: 0x558fc079a8e0
      1 : 123 (number)
      params : table: 0x558fc079a920
        terminal : urxvt (string)
        ntags : 5 (number)
        browser : firefox (string)
      rice : Openbox-based (string)
      scheme : Fleon (string)
      self : table: 0x558fc079a8e0
        1 : 123 (number)
        params : table: 0x558fc079a920
          terminal : urxvt (string)
          ntags : 5 (number)
          browser : firefox (string)
        rice : Openbox-based (string)
        scheme : Fleon (string)
        self : tabl

In [54]:
t2.self:params()

{ 123,[true] = 1,params = <function 1>,rice = "Openbox-based",scheme = "Fleon",self = <function 2>}

In [67]:
tostring(t)

"table: 0x558fc0781d40"